# Analyse de churn

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [3]:
#on importe les données
frame_churn=pd.read_csv("telecom.csv")

In [4]:
frame_churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 21 columns):
State             3333 non-null object
Account Length    3333 non-null int64
Area Code         3333 non-null int64
Phone             3333 non-null object
Int'l Plan        3333 non-null object
VMail Plan        3333 non-null object
VMail Message     3333 non-null int64
Day Mins          3333 non-null float64
Day Calls         3333 non-null int64
Day Charge        3333 non-null float64
Eve Mins          3333 non-null float64
Eve Calls         3333 non-null int64
Eve Charge        3333 non-null float64
Night Mins        3333 non-null float64
Night Calls       3333 non-null int64
Night Charge      3333 non-null float64
Intl Mins         3333 non-null float64
Intl Calls        3333 non-null int64
Intl Charge       3333 non-null float64
CustServ Calls    3333 non-null int64
Churn?            3333 non-null object
dtypes: float64(8), int64(8), object(5)
memory usage: 546.9+ KB


In [5]:
#on extrait pluslieurs lignes
frame_churn.iloc[5:15]

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
5,AL,118,510,391-8027,yes,no,0,223.4,98,37.98,...,101,18.75,203.9,118,9.18,6.3,6,1.70,0,False.
6,MA,121,510,355-9993,no,yes,24,218.2,88,37.09,...,108,29.62,212.6,118,9.57,7.5,7,2.03,3,False.
7,MO,147,415,329-9001,yes,no,0,157.0,79,26.69,...,94,8.76,211.8,96,9.53,7.1,6,1.92,0,False.
8,LA,117,408,335-4719,no,no,0,184.5,97,31.37,...,80,29.89,215.8,90,9.71,8.7,4,2.35,1,False.
9,WV,141,415,330-8173,yes,yes,37,258.6,84,43.96,...,111,18.87,326.4,97,14.69,11.2,5,3.02,0,False.
10,IN,65,415,329-6603,no,no,0,129.1,137,21.95,...,83,19.42,208.8,111,9.40,12.7,6,3.43,4,True.
11,RI,74,415,344-9403,no,no,0,187.7,127,31.91,...,148,13.89,196.0,94,8.82,9.1,5,2.46,0,False.
12,IA,168,408,363-1107,no,no,0,128.8,96,21.90,...,71,8.92,141.1,128,6.35,11.2,2,3.02,1,False.
13,MT,95,510,394-8006,no,no,0,156.6,88,26.62,...,75,21.05,192.3,115,8.65,12.3,5,3.32,3,False.
14,IA,62,415,366-9238,no,no,0,120.7,70,20.52,...,76,26.11,203.0,99,9.14,13.1,6,3.54,4,False.


In [6]:
#on vérifie le nombre de True / False
frame_churn["Churn?"].value_counts()

False.    2850
True.      483
Name: Churn?, dtype: int64

In [7]:
#on crée une variable binaire
frame_churn["churn"]=np.where(frame_churn["Churn?"]=="True.",1,0)

In [8]:
#on extrait les données
y=frame_churn["churn"]
x=frame_churn.select_dtypes([np.number]).drop("churn",axis=1)

In [9]:
x.shape

(3333, 16)

In [10]:
y.shape

(3333,)

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, KFold,train_test_split

In [12]:
#on crée un modèle de fôrets aléatoires, de SVM et de KNN
modele_rf=RandomForestClassifier(n_estimators=100)
modele_svm=SVC()
modele_knn=KNeighborsClassifier(n_neighbors=10)

In [13]:
#on crée une séparation pour la validation croisée
kf = KFold(n_splits=5)
kf.get_n_splits()

5

In [14]:
from sklearn.metrics import accuracy_score,confusion_matrix,recall_score,roc_curve

In [15]:
#on crée une fonction qui calcule l'accuracy par validation croisée
def ma_valid_croisee(modele,x,y,nb_fold=5):
    kf = KFold(n_splits=nb_fold,shuffle=True)
    accuracy_moy=0
    for train_index, test_index in kf.split(x):
        X_train, X_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        modele.fit(X_train,y_train)
        accuracy_moy += accuracy_score(y_test,modele.predict(X_test))
        
    accuracy_moy/=kf.get_n_splits()
    return accuracy_moy


In [16]:
#on applique cette fonction au modèle de KNN
ma_valid_croisee(modele_knn,x,y,7)

0.87669218630848555

In [17]:
#on applique cette fonction au modèle de SVM
ma_valid_croisee(modele_svm,x,y,7)

0.8550785093911778

In [18]:
#on applique cette fonction au modèle de Random forrest
ma_valid_croisee(modele_rf,x,y,7)

0.92409542433325542

In [21]:
#on sépare le jeu de données en apprentissage / test
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3)

In [22]:
modele_rf.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [23]:
#on affiche la matrice de confusion et l'accuracy
print(confusion_matrix(y_test,modele_rf.predict(x_test)))
print(accuracy_score(y_test,modele_rf.predict(x_test)))

[[846  11]
 [ 75  68]]
0.914


In [24]:
print(modele_knn.fit(x_train,y_train))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')


In [25]:
#on affiche la matrice de confusion et l'accuracy
print(confusion_matrix(y_test,modele_knn.predict(x_test)))
print(accuracy_score(y_test,modele_knn.predict(x_test)))

[[852   5]
 [118  25]]
0.877


In [26]:
modele_svm.fit(x_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [27]:
#on affiche la matrice de confusion et l'accuracy
print(confusion_matrix(y_test,modele_svm.predict(x_test)))
print(accuracy_score(y_test,modele_svm.predict(x_test)))

[[857   0]
 [143   0]]
0.857


On fait une recherche des meilleurs hyper-paramètres

In [33]:
from sklearn.model_selection import GridSearchCV

In [34]:
#on utilise comme indice de qualité le recall et on teste un nombre de voisins variables (2,4,6,8)
grid_model=GridSearchCV(KNeighborsClassifier(),
                        {"n_neighbors":[2,4,6,8]},scoring="recall",cv=5)

In [35]:
grid_model.fit(x,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_neighbors': [2, 4, 6, 8]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring='recall', verbose=0)

In [36]:
#on obtient le meilleur pramètre pour l'indice utilisé
print(grid_model.best_params_, grid_model.best_score_)

{'n_neighbors': 4} 0.219459188187
